In [1]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .enableHiveSupport()\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/04 13:25:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df_table = spark.read.format("jdbc") \
    .option("url", "jdbc:postgresql://metastore:5432/metastore") \
    .option("dbtable", "information_schema.tables") \
    .option("user", "jupyter") \
    .option("password", "jupyter") \
    .load()

In [4]:
df_table.show(3)

+-------------+------------+----------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+----------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|       DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|  DB_PRIVS|BASE TABLE|                        null|                null|                     null|      

In [12]:
df_table.count()

246

In [5]:
df_table.createOrReplaceTempView('df_table_v')

In [6]:
df_table.write.format("orc").saveAsTable('df_table_schema')

In [7]:
! hive -e "show create table df_table_schema"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = d7bb266f-9048-49f7-8659-4f4f8c28c31d

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = cffd182d-669c-4f2b-9a14-b673544abb90
OK
CREATE TABLE `df_table_schema`(
  `table_catalog` string, 
  `table_schema` string, 
  `table_name` string, 
  `table_type` string, 
  `self_referencing_column_name` string, 
  `reference_generation` string, 
  `user_defined_type_catalog` string, 
  `user_defined_type_schema` string, 
  `user_

In [8]:
! hive -e " CREATE TABLE tables ( \
   table_catalog string, \
   table_schema  string, \
   table_name  string, \
   table_type  string, \
   self_referencing_column_name  string, \
   reference_generation  string, \
   user_defined_type_catalog  string, \
   user_defined_type_schema  string, \
   user_defined_type_name  string, \
   is_insertable_into  string, \
   is_typed  string, \
   commit_action  string) \
ROW FORMAT SERDE \
  'org.apache.hadoop.hive.ql.io.orc.OrcSerde' \
LOCATION \
  'hdfs://master/user/hive/warehouse/tables' \
  "

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 9de35d91-43f5-4049-8ebe-33ed3d87d0da

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = ec0212b4-8d16-4048-81cd-0d8e8ffcf59f
FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. AlreadyExistsException(message:Table hive.default.tables already exists)


In [13]:
! hive -e "insert into tables (select * from df_table_v)"

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/opt/hive/lib/log4j-slf4j-impl-2.17.1.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/opt/hadoop/share/hadoop/common/lib/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]
Hive Session ID = 357ab0d8-af43-4c79-ba78-3c347d8e4f28

Logging initialized using configuration in jar:file:/opt/hive/lib/hive-common-3.1.3.jar!/hive-log4j2.properties Async: true
Hive Session ID = 18446d19-1a62-4042-9205-df18cff64f53
Query ID = jupyter_20250104133618_62d8970a-f012-4c1e-aabb-dbb7cafabcc9
Total jobs = 3
Launching Job 1 out of 3
Number of reduce tasks determined at compile time: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit t

In [14]:
spark.sql('insert into tables (select * from df_table_v)')

++
||
++
++



In [15]:
spark.sql('select * from tables').show(3)

+-------------+------------+----------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|table_catalog|table_schema|table_name|table_type|self_referencing_column_name|reference_generation|user_defined_type_catalog|user_defined_type_schema|user_defined_type_name|is_insertable_into|is_typed|commit_action|
+-------------+------------+----------+----------+----------------------------+--------------------+-------------------------+------------------------+----------------------+------------------+--------+-------------+
|    metastore|      public|       DBS|BASE TABLE|                        null|                null|                     null|                    null|                  null|               YES|      NO|         null|
|    metastore|      public|  DB_PRIVS|BASE TABLE|                        null|                null|                     null|      